In [1]:
# 기본 패키지
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import datetime as dt

# 시각화
import seaborn as sns
import matplotlib.pyplot as plt
## 이미지에서 한글이 나오게 하는 설정
plt.rcParams["figure.figsize"] = (8, 6)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
%matplotlib inline

## 야후 파이낸스 패키지 ## pip install yfinance
import yfinance as yf

## 연준 패키지 ## pip install fredapi
from fredapi import Fred
fred = Fred(api_key='cfb4f49f5c1a9396f671b8049d992e56')
## fred = Fred(api_key='MY API KEY')
## FED API KEY 발급받기
## https://yenpa.tistory.com/65

In [2]:
## 사전 정의한 함수 모음
import graphModule as gm
import myfuncs as mf

## DataLoad

In [3]:
## 필수 데이터

## 사전에 획득한 금, 다우존스배당지수
gold = pd.read_csv(f"gold.csv", encoding='utf-8').set_index('Date')[['Close']].rename(columns={'Close':'Gold'})
gold.index = pd.to_datetime(gold.index)
div = pd.read_csv(f"div.csv", encoding='utf-8').set_index('Date')[['Close']].rename(columns={'Close':'Div'})
div.index = pd.to_datetime(div.index)
## DFF: Federal Funds Rate, DTB3: 3개월 미국 국채 금리, DGS2: 2년 미국 국채 금리, DGS10: 10년 미국 국채 금리, DGS20: 20년 미국 국채 금리
ds20 = fred.get_series('DGS20')
ds10 = fred.get_series('DGS10')
ds2 = fred.get_series('DGS2').fillna(method='pad')
tb3 = fred.get_series('DTB3').fillna(method='pad')
dff = fred.get_series('DFF').fillna(method='pad')
## rec = fred.get_series('USREC').fillna(method='pad') ## 월간 경기침체 데이터 ## 별도 수집 데이터로 대체

## 기본지수 데이터
## ^IXIC : 나스닥종합주가지수, ^GSPC: S&P500지수, ^DJI: 다우존스지수, GC=F : 금선물지수
tickers = ['^IXIC', '^GSPC', '^DJI', 'GC=F']
df_ref = yf.download(tickers, ignore_tz = True, auto_adjust=True)
df_ref = df_ref['Close']

## 75년 이전에는 Gold 데이터가 없음
df_ref = df_ref['1975':]
df_ref = df_ref.fillna(method = 'pad')
df_ref['Div'] = div['Div'].copy()
df_ref['Gold'] = gold['Gold'].copy()
df_ref['DGS10'] = ds10
df_ref['DGS20'] = ds20
df_ref['DGS2'] = ds2
df_ref['DTB3'] = tb3

[*********************100%%**********************]  4 of 4 completed


## Preprocessing

In [10]:
## Imputation
df_ref_imp = mf.imputation(df_ref, '^GSPC', '^DJI') ## S&P500를 기준으로 다우존스 지수 보간
df_ref_imp = mf.imputation(df_ref_imp, '^DJI', 'Div') ## 다우존스 지수를 기준으로 배당지수 보간
df_ref_imp = mf.imputation(df_ref_imp, 'DGS10', 'DGS20', log_diff=False)
df_ref_imp = mf.imputation(df_ref_imp, 'DGS20', 'DGS10', log_diff=False)
df_ref_imp = mf.imputation(df_ref_imp, 'DGS10', 'DGS2', log_diff=False)
df_ref_imp = mf.imputation(df_ref_imp, 'DGS2', 'DTB3', log_diff=False)
df_ref_imp = mf.imputation(df_ref_imp, 'GC=F', 'Gold') ## 금 선물 가격을 기준으로 금 가격 보간
## column selection
df_ref_imp = df_ref_imp[['^GSPC', '^DJI', '^IXIC', 'Div', 'Gold', 'DTB3', 'DGS2', 'DGS10', 'DGS20']]

0.9184937283083807
0.8698779253276526
0.8862974392044377
1.01534223507469
0.6602857065285672
0.7090803399458092
0.9953424146502147


In [24]:
df_ref_scaled = df_ref_imp.copy()
_cols = ['^GSPC', '^DJI', '^IXIC', 'Div', 'Gold']
df_ref_scaled[_cols] = df_ref_scaled[_cols]/df_ref_scaled[_cols].iloc[0] * 100
df_ref_scaled.columns = ['S&P500', 'DowJones', 'NASDAQ', 'Div', 'Gold', 'DTB3', 'DGS2', 'DGS10', 'DGS20']
df_ref_scaled['Cash'] = (df_ref_scaled['DTB3']/100 * 1/252 + 1).shift(1).fillna(1).cumprod() * 100

In [25]:
df_ref_scaled

,S&P500,DowJones,NASDAQ,Div,Gold,DTB3,DGS2,DGS10,DGS20,Cash
Date,,,,,,,,,,
1975-01-02,100.000000,100.000000,100.000000,100.000000,100.000000,6.900000,6.853275,7.420000,7.940000,100.000000
1975-01-03,100.683462,100.627582,100.873145,100.545697,100.000000,6.760000,6.859878,7.430000,7.920000,100.027381
1975-01-06,101.196065,101.098045,101.713346,100.954485,99.828277,6.700000,6.833466,7.390000,7.890000,100.054214
1975-01-07,101.124866,101.032710,101.960459,100.897731,99.255867,6.510000,6.826863,7.380000,7.890000,100.080815
1975-01-08,99.729457,99.751480,101.317956,99.783783,101.545507,6.540000,6.833466,7.390000,7.860000,100.106670
...,...,...,...,...,...,...,...,...,...,...
2025-01-24,8687.512378,7195.149891,32873.641731,15373.692219,1523.393417,4.210000,4.270000,4.630000,4.910000,864.715391
2025-01-27,8560.842229,7242.010766,31864.629053,15460.753174,1501.663406,4.200000,4.170000,4.530000,4.820000,864.859853
2025-01-28,8639.754956,7264.163203,32510.032283,15501.883765,1517.660774,4.200000,4.190000,4.550000,4.840000,865.003996
